# Problem set 3

Most of the exercises here will require you to search for functions on Google as it is important to learn how to find answers to your questions. It is more challenging to find answers in Julia than in Python since it is a relatively new language. However, the Julia community is very active and helpful, so you should be able to find answers to most of your questions.

## Problem 1: Bouncing ball

Using the callback functionality of DifferentialEquations.jl, implement the bouncing ball physical system:
$$
\dot x = v\\
\dot v = -g - \gamma v
$$
with $g=10$ , $\gamma=0.99$, $x_0 = 1$, $v_0 = 0$. The ball bounces elastically off the ground, i.e. $v \leftarrow -v$. Implement this problem for elastic collisions (elastic collisions preserve velocity measure), and plot the time evolution of x and v versus time.

* Animate the ball bouncing for 10 seconds with a vector that represents the velocity of the ball at each time step. Save the animation in a file called `Bouncing_Ball.mp4`.

**Hint:** whenever the ball reaches the level x = 0 , its velocity should be reversed.

In [1]:
using Pkg
Pkg.activate(".")
using GLMakie
using OrdinaryDiffEq, StaticArrays
using DynamicalSystems

  Activating project at `c:\Users\USAID\Documents\2023-2024\phys 218\NonLinearDynamics\Julia Practice`


In [3]:
# Fill in the initial conditions, constants and time span (0->10s)
t = (0.0, 10.0)
p = [10, 0.99]
u0 = [1, 0];

In [16]:
# Define the ODE system as a function with the constraint that the ball should Bounce when it hits the ground x = 0 
function Ball(u)
    du1 = u[2]
    du2 = -10-0.99*u[2]
    return [du1, du2]
end

condition(u,t,integrator) = u[1] == 0

function affect!(integrator)
    integrator.u = (u[1], -u[2])
end

cb = ContinuousCallback(condition,affect!)

ContinuousCallback{typeof(condition), typeof(affect!), typeof(affect!), typeof(SciMLBase.INITIALIZE_DEFAULT), typeof(SciMLBase.FINALIZE_DEFAULT), Float64, Int64, Rational{Int64}, Nothing, Int64}(condition, affect!, affect!, SciMLBase.INITIALIZE_DEFAULT, SciMLBase.FINALIZE_DEFAULT, nothing, SciMLBase.LeftRootFind, 10, Bool[1, 1], 1, 2.220446049250313e-15, 0, 1//100)

In [17]:
# Solve the ODE system using the ODE solver Vern9()
prob = ODEProblem(Ball,u0,t)
sol = solve(prob;alg=Vern9(), callback = cb);

SciMLBase.NoMethodsError: No methods were found for the model function passed to the equation solver.
The function `f` needs to have dispatches, for example, for an ODEProblem
`f` must define either `f(u,p,t)` or `f(du,u,p,t)`. For more information
on how the model function `f` should be defined, consult the docstring for
the appropriate `AbstractSciMLFunction`.

Offending function: f

In [ ]:
# Get x, v, and t from the solution
xs = sol[1,:]
vs = sol[2,:];

In [ ]:
# Plot the trajectory of the ball and the velocity of the ball as a function of time
fig = Figure(size=(800,800))
ax = Axis3(fig[1, 1])
lines!(ax, xs, vs, t, color = :blue)
ax.azimuth = 90
fig

In [ ]:
# Plot them using interpolations to get a smooth curve

In [ ]:
# Here insert the code to get the animation 

## Problem 2: Distribution quantile

To find the q-th quantile of a distribution, one can use Newton's method:
$$
\theta_{n+1} = \theta_n - \frac{\text{cdf}(\theta_n) - q}{\text{pdf}(\theta_n)}
$$
where $\theta_n$ converges to the value of the q-th quantile. cdf and pdf are the cumulatide and probability density functions respectively.

Write a generic function that implements the algorithm which calculates the qth quantile of any UnivariateDistribution in `Distributions.jl`, and test your result against the `quantile(d::UnivariateDistribution, q::Number) `function from `Distributions.jl` using the following distributions: `[Gamma(5, 1), Normal(0, 1), Beta(2, 4)].`

use $\theta_0 = \text{median}(\text{distribution})$

In [ ]:
using ...

In [ ]:
# Your function to calculate quantile

In [ ]:
# Compare the two methods 

## Problem 3: Plotting subsets of a dataframe

Using the `DataFrames` package, load the `iris` dataset. with the following code:

In [ ]:
using DataFrames, CSV
iris = DataFrame(CSV.File(
    joinpath(dirname(pathof(DataFrames)), 
    "../docs/src/assets/iris.csv")
));
iris[1:5,:]

This dataset has various flower species (column :Species). For every species create a 1x2 figure with the following plots:
* [1,1] = scatter plot of SepalLength vs SepalWidth.
* [1,2] = scatter plot of PetalLength vs PetalWidth.

For each of these sub-scatter plots, calculate and print the Pearson correlaton coefficient.

In [ ]:
using ...

In [ ]:
# Write all the code here 

## Problem 4: DataFrame of chaos
Define the Roessler system as a DynamicalSystem:
$$
\dot x = -y - z\\   
\dot y = x + a y\\
\dot z = b + z(x - c)
$$
with initial condition ones(3). Loop over all three of the following parameter ranges:

as = 0.15:0.025:0.25

bs = 0.15:0.025:0.25

cs = 4:0.1:6.0

and for each parameter combination calculate:
* (1) the Lyapunov spectrum using lyapunovspectrum 
* (2) the Shannon entropy of the system's trajectory. For the entropy, simulate a trajectory of the system with sampling time Δt=0.1 for T = 1000.0 and use ε=0.1 for the box size when calling `DynamicalSystems.entropy(ValueHistogram(ε), X)` to estimate the entropy. For changing parameters you have to use `set_parameter!`. 
* Remember that you can push entries to a DataFrame row-by-row using `push!(dataframe, (a = a, b = b, ...))`.
* Collect this analysis into a dataframe, with columns `a, b, c, λ1, λ3, H` with `λ1, λ3` the first and last elements of lyapunovspectrum

Then use the querying framework to query into this dataframe and do the following tasks:
* #### find chaotic parameter sets
    Select all values a, b, c where λ1 > 0.01 (which indicates chaotic dynamics).
* #### $\lambda$ vs H
    For all chaotic cases, select the λ1, H values and then perform a scatter plot of these two.
* #### heatmap of H
    For `a` fixed to 0.2, plot a heatmap of the value of `H` with axis the values of `c` and the values of `b`. Provided that you have a dataframe with columns `:b, :c, :H`, then these commands:

In [ ]:
unstacked = unstack(df, :b,  :H; renamecols = (x -> "H for b=$(x)"))
heat = Matrix(unstacked[:, Not(:c)])


will create a matrix with the values of H where the row index is the values of c and the column index is the values of b.

Hint: you can initiallize an empty dataframe with df = DataFrame(), and then start pushing into it arbitrary named tuples like push!(df, (a = 0.5, b = 0.5)).

Hint 2: this all sounds like a lot of computation, but if you do the exercise correct running all code should not take more than 10 seconds.

In [ ]:
using ...

In [ ]:
# Define the Roessler system as a function

In [ ]:
# Define the initial conditions

In [ ]:
# Fill in the DataFrame with all the conditions

In [ ]:
# Get the Chaotic parameters

In [ ]:
# Scatter plot λ1 vs H

In [ ]:
# fix a and get the new DataFrame

In [ ]:
unstacked = unstack(fixa, :b,  :H; renamecols = (x -> "H for b=$(x)"))
heat = Matrix{Float64}(unstacked[:, Not(:c)]);

In [ ]:
# Plot the heatmap